In [21]:
from getpass import getpass
import os
import numpy as np
import tensorflow as tf


In [22]:
!pip install -qU \
  pinecone-client \
  sentence-transformers

In [3]:
!wget -O businessexport-20230626T124852Z-001.zip https://github.com/lrmillennium/businessrecommender/raw/main/businessexport-20230626T234723Z-001.zip

--2023-06-27 00:06:00--  https://github.com/lrmillennium/businessrecommender/raw/main/businessexport-20230626T234723Z-001.zip
Resolving github.com (github.com)... 140.82.112.3
Connecting to github.com (github.com)|140.82.112.3|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://raw.githubusercontent.com/lrmillennium/businessrecommender/main/businessexport-20230626T234723Z-001.zip [following]
--2023-06-27 00:06:01--  https://raw.githubusercontent.com/lrmillennium/businessrecommender/main/businessexport-20230626T234723Z-001.zip
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.111.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 22681252 (22M) [application/zip]
Saving to: ‘businessexport-20230626T124852Z-001.zip’

businessexport-2023 100%[===================>]  21.63M   130MB/s  

In [23]:
import os
os.listdir ('.')
from zipfile import ZipFile
with ZipFile ('businessexport-20230626T124852Z-001.zip', 'r') as f :
  f.extractall ()
os.listdir('.')

['.config',
 'businessexport-20230626T124852Z-001.zip',
 'businessexport',
 'sample_data']

In [24]:
os.environ['API_KEY'] = getpass()

··········


In [25]:
os.environ['API_ENV'] = getpass()

··········


In [38]:
# get the pipecone client
import os
import pinecone

pinecone.init(
    api_key = os.getenv('API_KEY'),
    environment=os.getenv('API_ENV')
)
index_name = 'yelpbusiness-search'
index = pinecone.GRPCIndex(index_name)
index.describe_index_stats()

{'dimension': 768,
 'index_fullness': 0.7,
 'namespaces': {'': {'vector_count': 150346}},
 'total_vector_count': 150346}

In [39]:
# get the model
from sentence_transformers import SentenceTransformer
import torch

device = 'cuda' if torch.cuda.is_available() else 'cpu'
if device != 'cuda':
    print(f"You are using {device}. This is much slower than using "
          "a CUDA-enabled GPU. If on Colab you can change this by "
          "clicking Runtime > Change runtime type > GPU.")

#model = SentenceTransformer('all-MiniLM-L6-v2', device=device)
model = SentenceTransformer('all-mpnet-base-v2', device=device)

You are using cpu. This is much slower than using a CUDA-enabled GPU. If on Colab you can change this by clicking Runtime > Change runtime type > GPU.


In [46]:
query = "italian restaurants in tampa?"

# create the query vector
xq = model.encode(query).tolist()

# now query
xc = index.query(xq, top_k=20, include_metadata=True)
xc.to_dict()['matches']

{'id': '7O6s2C-6X_qMcH29mhozLw',
 'score': 0.7555567,
 'values': [],
 'sparse_values': {'indices': [], 'values': []},
 'metadata': {'text': 'Da Giuseppe Trattoria with the location of 5915 Memorial Hwy, in the city of Tampa, in the state of FL with the zip code of 33615 in the business categories of Restaurants, Italian'}}

In [59]:
model = tf.saved_model.load("businessexport")

In [75]:
test_ratings = {}
test_business_id = []
iddescription = {}
for item in xc.to_dict()['matches'] :
  test_business_id.append(item['id'])
  iddescription[item['id']] = item['metadata']['text']
test_business_id


['7O6s2C-6X_qMcH29mhozLw',
 '_FIBtjR9tNNcwfpsEtFABg',
 '_A1Dr1Ms8PUuAif7bR_Zcw',
 'Pm6qD9G_xWLoV24YIN8gwQ',
 'TqztcztbTJVaUj_3XLjFdg',
 'DVG3oqNuDZbTDe3zCUZsng',
 'YVvm1Q3ir5TET0f66FOX_g',
 'E8vPptotPwJ1eBGu1Vfmyg',
 'FFV8_b-ec5Z0E3xgvkVxKw',
 '1IjHo5kaSdFxJkJotU4O7A',
 'zjqh_qoBS1BWVSbC51BNjw',
 'ihxXIksbCZ9fTirbFQF0NQ',
 '0g7s_CI35bXAHo_SATlHBQ',
 'X-IWPL5GY6ejEciBGMVK9A',
 'RiC_-68qxtDJqiIs5mRR6g',
 '5cz1XbgalpQ89gsNQwneVQ',
 'LJzWJcPUZl-W240IWHELcw',
 'zEun4V7It5MWTIvqZmIlpA',
 'xEvV6_CSA9pJAVK0i9UDWg',
 'p3-FsWgkrRlT18Mw5Z1tHg']

In [76]:
for business_id in test_business_id:
  test_ratings[business_id] = model({
      "user_id": np.array([b'plobBsTtVUODb353xxFT_g']),
      "business_id": np.array([business_id])
  })

print("Ratings:")
for business_id, score in sorted(test_ratings.items(), key=lambda x: x[1], reverse=True):
  print(f"{business_id}: {score} : {iddescription[business_id]}")

Ratings:
FFV8_b-ec5Z0E3xgvkVxKw: [[3.7617939]] : Matteo Trattoria & Pizzeria with the location of 2402 S MacDill Ave, in the city of Tampa, in the state of FL with the zip code of 33629 in the business categories of Restaurants, Italian
zEun4V7It5MWTIvqZmIlpA: [[3.738233]] : DeVito's Italian Specialties with the location of 200 N Tampa St, Ste G 115, in the city of Tampa, in the state of FL with the zip code of 33602 in the business categories of Italian, Restaurants
0g7s_CI35bXAHo_SATlHBQ: [[3.7355702]] : Bocelli Modern Italian with the location of 5427 Bayshore Blvd, in the city of Tampa, in the state of FL with the zip code of 33611 in the business categories of Italian, Restaurants, Pizza
ihxXIksbCZ9fTirbFQF0NQ: [[3.7335963]] : Italy Bottega with the location of 1045 Central Ave, in the city of St. Petersburg, in the state of FL with the zip code of 33705 in the business categories of Wine Bars, Specialty Food, Italian, Bars, Restaurants, Nightlife, Beer Bar, Food, Desserts, Delis
